# 🧠 Sprint 2 — Pinecone Embeddings Search

This notebook replicates the FAISS-based workflow using **Pinecone** for vector search.

We'll use the same embedding model (`all-MiniLM-L6-v2`) and chunking logic.


In [ ]:
# Install dependencies
!pip install pinecone sentence-transformers tqdm

In [ ]:
import pinecone
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import numpy as np

# Initialize model
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# 🔑 Initialize Pinecone (new SDK)
from pinecone import Pinecone, ServerlessSpec

api_key = "pcsk_78NBUV_CMTsLskzpvRZLAi6Xvrh3eERC3QpFpZAhRUF9S9yTXWZAGvXBkBHN9rUCEEmtBx"
pc = Pinecone(api_key=api_key)

index_name = "sprint2-embeddings"

# Create index if it doesn't exist
if index_name not in [index.name for index in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# Connect to index
index = pc.Index(index_name)
print("✅ Pinecone index ready and connected!")


✅ Pinecone index ready and connected!


In [ ]:
# 📄 Example text (replace this with your data)
text = """
Abstract: This paper examines the ethical issues surrounding drone warfare in the Russia-Ukraine war. It analyzes how engineering choices in unmanned aerial systems (UAS) like the low-cost Iranian Shahed-136 loitering munition and the sophisticated Turkish Bayraktar TB2 reflect trade-offs in cost, autonomy, and reliability, and how these design choices pose moral challenges. The study reviews how engineers' responsibilities (per codes like NSPE) intersect with the deployment of lethal autonomous weapons. It also assesses the civilian impact of nightly drone barrages – from infrastructure damage to psychological trauma – using case examples (e.g., Kyiv and Kherson strikes). Key findings highlight that cheap, kamikaze drones enable mass attacks that often violate the just-war principles of distinction and proportionality, causing widespread fear and sleep deprivation among civilians . The paper concludes that engineers must carefully weigh the public welfare in designing military systems, and that unchecked drone proliferation risks eroding public trust in technology and international law. Introduction:
Drone warfare has become a defining aspect of modern combat, raising profound ethical questions in engineering. Small unmanned aerial vehicles (UAVs) now conduct surveillance and precision strikes that were once the sole domain of manned aircraft. Their availability and effectiveness have significantly altered conflict: analysts note that both sides in Ukraine have deployed thousands of small drones for intelligence, reconnaissance, and direct attack . The war in Ukraine thus exemplifies the new era of drone warfare and its ethical implications. Engineers designing these systems face dilemmas such as balancing mission autonomy against accountability, and optimizing cost-efficiency at the risk of indiscriminate use.
The importance of this topic to engineering ethics lies in the tension between professional duties and wartime imperatives. Codes of ethics (e.g. the NSPE Code) insist engineers "hold paramount the safety, health, and welfare of the public" , yet military drones are explicitly built to kill. This study will explore how engineering decisions in drone design and deployment create moral hazards, and examine specific case studies from the Russia-Ukraine conflict.
The objectives are to:
•	outline the technical and ethical trade-offs in modern combat drones
•	Compare different drone models in terms of  design and use
•	analyze documented drone strikes in Ukraine and their humanitarian effects
•	reflect on the broader responsibilities of engineers in armed conflict.
With added examples and deeper analysis, this paper seeks to provide a nuanced understanding of drones' dual nature as tools of progress and destruction.
Technical and Ethical Dimensions of Drone Warfare:
 Design Trade-offs:Shahed-136 vs. Bayraktar TB2
 Combat drones vary widely in complexity, cost, and capability. A clear illustration is the difference between the Iranian-made Shahed-136 and the Turkish Bayraktar TB2. The Shahed-136 is a kamikaze "loitering munition" – a single-use drone carrying a warhead for one-way missions .It is small (length ≈3.5 m, wingspan ≈2.5 m, weight ~200 kg) with a crude autopilot (inertial/GPS guidance) and a ~50 kg explosive payload . In contrast, the Bayraktar TB2 is a much larger reusable UAV (length 6.5 m, span 12 m, MTOW 700 kg) with sophisticated avionics. It carries 4 smart guided bombs or missiles (total 150 kg payload) and can fly for 27 hours with live human control at up to 222 km/h . (fig1)Relative sizes of the Iranian Shahed-136 (left) and the Turkish Bayraktar TB2 (right) drones. The Shahed's small, delta-wing design and limited range (order 1,000–2,500 km) contrasts with the much larger, long-endurance TB2.
Drone	Shahed 136	Bayraktar TB2
Manufacturer	HESA	Baykar
Length	3,5 m	6,5 m
Wing span	2,5 m	12 m
MOTW	200 kg*	650 kg
Speed	185 km/h	130 km/h
Range	2.500 km*	150 km
Engine	50 hp	110 hp
Payload	40 kg	140 kg
Takeoff	By platform	Runway
Price	$ 20,000	$ 1,000,000
Figure 2
These design choices reflect different priorities. The Shahed is extremely cheap (reported production cost tens of thousands of USD) , allowing Russia to launch salvos of dozens each night. This swarming strategy saturates air defenses but sacrifices accuracy and reusability. By contrast, a Bayraktar costs on the order of $1–5 million , is human-piloted, and must return after each mission. Its advanced sensors and precision weapons enable targeted strikes with minimal collateral damage when used properly, but its high cost and vulnerability to air defenses limit the number that can be deployed. Engineers must trade cost vs. precision and autonomy vs. control. High autonomy (as in the Shahed's simple AI pilot) can permit stand-off attacks without risking pilots but reduces accountability and can lead to more civilian hits if guidance fails. In both designs, reliability is also an ethical issue: a kamikaze drone that misses its target may crash indiscriminately, and a multi-ton UAV malfunction could fall on civilians. The necessity to protect public welfare (as per engineering ethics codes) clashes with the engineering goal to maximize a weapon's effectiveness.(fig2) shows a comparison between Shahed-136 and Bayraktar TB2
Control and Autonomy
Modern drones often incorporate various levels of autonomy. A key question is how much decision making the drone can do independently. The Shahed-136, for example, uses a pre-programmed flight plan to loiter and descend on a target, with only a basic inertial/GPS guidance . The TB2, by contrast, is controlled in real-time by a pilot via encrypted datalink, allowing human judgment in target selection . The increasing integration of AI (e.g., for obstacle avoidance or automated target recognition) raises new ethical concerns: Who is responsible if an autonomous system misidentifies a civilian structure as a military target? The issue of accountability looms large – current engineering codes expect transparency and verification, but autonomous drones can obscure decision chains. Some defense analysts warn that technology often outpaces regulation, noting that "drones have become more capable, but ethical and regulatory frameworks have lagged" . Engineering professionals thus face the dilemma of innovating in warfare technology while ensuring human oversight and compliance with international law.
Engineers' Responsibilities and Codes of Ethics
Engineers involved in military projects must reconcile their work with professional ethical standards. The NSPE Code of Ethics explicitly states that "Engineers shall hold paramount the safety, health, and welfare of the public." . Designing lethal systems seems at odds with this canon, but engineers often justify weapons development under national security imperatives. However, even military engineers are bound to avoid deceptive or malicious acts and to refuse assignments that endanger public life without proper safeguards. For example, if an engineer learns that a drone design is likely to fail dangerously or breach the laws of armed conflict, the NSPE code would require them to raise concerns or withdraw from the project . Ethical guidelines from IEEE and IET similarly emphasize human oversight: weapons should not operate with full autonomy to kill without human judgment.
In practice, some argue engineers cannot be held personally liable if a lawful weapon is misused. Nevertheless, ethical training encourages engineers to consider the downstream consequences of their designs. If a drone's sensors are insufficient to distinguish combatants from civilians, engineers must recognize the risk of collateral damage. Engineering societies increasingly call for "caution" in weapon automation – noting that "drones pose serious ethical dilemmas around how, and whether, to regulate" their use . By applying professional codes, engineers must ensure their systems include fail-safe and respect targeting protocols (e.g., the Geneva Conventions' principle of distinction), even when operating in war. This means rigorous testing, honest reporting, and possibly whistleblowing if safety is compromised.
Civilian Impact and Broader Consequences
The deployment of drones in Ukraine has had severe humanitarian consequences. Though proponents claim precision, in practice both Russian and Ukrainian drone strikes have hit civilian areas. For instance, in January 2025 the UN reported short-range drones killed more Ukrainians than any other weapon: 27% of civilians killed (38 out of 139) and 30% of injuries (223 out of 738) in that month were from drone strikes as shown in fig(3). These "FPV" drone bombs, launched by operators on the ground, frequently struck cars, streets, and other public places. The Head of the UN Human Rights Mission in Ukraine noted, "Short-range drones now pose one of the deadliest threats to civilians in frontline areas.". Likewise, a UN report documented a single Russian drone strike on 17 May 2025 that killed 9 evacuees in a civilian minibus

Figure 3

Persistent drone barrages also inflict psychological trauma. Civilians describe almost nightly air-raid sirens as "bombing of sleep" – people cannot rest, leading to widespread insomnia and PTSD risks . Mental health professionals in Ukraine warn that sleep deprivation from constant drone attacks "weakens immune systems and raises the risk of long-term illnesses" . For example, one journalist notes Ukrainians have developed hypersensitivity: ordinary noises trigger panic, and many now live in "feigned arousal" mode . A 2024 study found that regions exposed to frequent air alarms (rockets and drones) showed significantly higher PTSD and sleep disturbance than quieter areas . In effect, cheap drones do not just destroy structures; they erode morale and well-being far from the front lines.
Infrastructure damage is another ethical concern. Reports from Ukraine's civil defense indicate that drones have struck schools, power plants, and apartment blocks. For instance, the April 2025 drone volley on Kyiv shattered windows of municipal buildings, injuring dozens and damaging civilian property . In Kherson city (partially occupied), a January 2025 strike on a passenger bus killed a man and injured nine , and local officials note such incidents occur "almost daily" . President Zelenskiy publicly called the April 2025 Marhanets bus hit (9 killed) "a deliberate war crime" . These episodes show that drone operators – and by extension the engineers who enable them – have a responsibility to consider foreseeable civilian harm under just-war norms. Collateral damage from falling drone wreckage (especially when thousands of pounds of debris rain down) can be severe and is largely unavoidable with loitering munitions.
Ethical Implications for Society and Future Warfare
Widespread drones also affects public trust in technology and warfare. When people see autonomous or remotely piloted weapons causing civilian suffering, they may lose faith in institutions and engineers who develop such systems. Communities report distrust not only of the warring parties but of any external actors promoting drone technology. Moreover, the success of drones in Ukraine has spurred a global arms race: dozens of countries are now rapidly expanding their drone fleets. Ethically, this raises the prospect of future conflicts being fought by increasingly automated means – a trend that demands robust oversight. Analysts warn that without new norms, the convenience of drone strikes could lower the bar for entering conflicts . Indeed, one dilemma is whether removing pilot risk makes leaders more prone to launch attacks. If the human cost to one's own forces is negligible, will governments bypass diplomacy more readily? Engineering ethicists worry that "the perceived safety for operators lowers the threshold for war," potentially leading to more frequent or prolonged conflicts .
The combination of low cost and high lethality of drone swarms is creating a new paradigm. Combatants and civilians alike see their world transformed by night-time drone swarms. Society must ask: should restrictions be placed on fully autonomous swarms? Should there be an international treaty limiting kamikaze drones? Engineers, who often champion innovation, must also engage in public and policy dialogues to help shape such regulations in line with humanitarian principles. In the meantime, they must embed ethical safeguards in design (e.g., programming strict no-fire zones, implementing human-in-the-loop controls, and ensuring transparency in target selection algorithms) to uphold the profession's duty to the public.

Case Study: Drone Operations in the Russia Ukraine War This section examines specific drone attacks during the current conflict and the ethical issues they illustrate.
Kyiv and Major Cities: On 24 April 2025, Russia launched a massive, combined missile-and drone assault on Kyiv. U.S. officials reported "at least 12 people" killed and about 90 wounded . Missiles destroyed buildings, but several deaths were caused by drones striking near civilian areas. Emergency services combed rubble for hours, illustrating how even precision tools failed to avoid urban casualties. The scale of this attack – the largest on Kyiv in 2025 – drew rare international condemnation. U.S. President Trump himself warned Russia to "STOP" after hearing of "not necessary" strikes on civilians . This incident underscores the ethical question of proportionality: were these drone strikes aimed exclusively at military infrastructure, or did they recklessly endanger non-combatants?
Public Transport: Drones have repeatedly targeted buses. In Kherson city (January 2025), a Russian drone hit a civilian coach, killing a 49-year-old man and wounding nine . The governor reported such strikes occur "almost daily" in frontline regions. In Dnipropetrovsk region (April 2025), a Russian attack on the city of Marhanets struck a mining workers' bus on a highway, killing nine and injuring nearly fifty . President Zelenskiy decried this as a war crime since the bus was an "ordinary civilian object" . Engineers who build targeting systems must ask: could improve sensor fidelity or stricter verification have prevented these tragedies? The likely answer is yes – human operators rely on intelligence cues. If engineers do not design robust friend-vs-foe identification, even an accurate weapon can become ethically compromised.
Battlefield Surveillance: On the military side, Ukrainian forces have used Bayraktar TB2 drones as recon assets. These drones have scouted Russian troop positions and even struck isolated armored vehicles far behind enemy lines . For example, one open-source analysis credits small Turkish drones with destroying dozens of Russian tanks and supplies by providing real time targeting data . Ethically, surveillance drones entail fewer direct harms than attack drones, but still pose questions: do engineers ensure their cameras respect privacy when used in civilian areas? Should drone software be configured to disable live streaming when missiles are absent? While these issues have not been as prominent in wartime reporting, they reflect the same principle: engineers should integrate constraints to protect non-targeted people whenever possible. 5 In sum, the Ukraine case studies reveal that despite advanced guidance, many drone strikes still hit civilians. As seen in figure below, Russia increasingly relies on one-way attack drones to sustain its attack on Ukrainian critical infrastructure and political centers consistent with their concept of noncontact war and using long-range precision strike assets. Every night, these systems cause millions of Ukrainians to head to bomb shelters and mobile air

Figure 4
This highlights the engineer's ethical duty to minimize harm. Even if a weapon functioned "as designed," its deployment context can make it unethical. Therefore, engineers should anticipate misuse and error on the side of caution, advocating for strict adherence to the laws of armed conflict in any drone system they produce.
Conclusion :
Drone warfare in Ukraine exemplifies the complex ethical challenges of engineering modern weapons. This case study shows that design decisions – cost, autonomy, reliability – have moral weight. The Iranian Shahed-136 and Turkish Bayraktar TB2 represent two ends of the spectrum: one a cheap mass produced "suicide" UAV, the other an expensive precision drone. Both have strategic value, but their use raises engineers' ethical responsibilities. To prevent civilian suffering, engineers must embed safeguards, maintain human oversight, and heed professional codes requiring public safety to be paramount .
Key findings include: (1) Cost-effective drones like the Shahed enable mass attacks that overwhelm defenses but often violate the principles of distinction, resulting in high civilian casualties . (2) Precision drones like the TB2 can be more discriminating but are costly and vulnerable, potentially limiting their deployment. (3) Psychological harm from nightly drone barrages imposes long-term burdens on society . (4) Engineer ethics require balancing military objectives with minimization of harm; according to NSPE, engineers must prioritize public welfare, even when working on defense projects .
Readers should take away that engineering in warfare is never neutral: every technical choice has social and moral implications. As autonomous weapons proliferate, engineers must engage proactively in ethical reflection and policymaking. Ensuring that drone technology serves defensive security goals without undermining human values will require strict professional vigilance. Ultimately, the future of autonomous warfare will depend on the engineering community's commitment to ethical principles, just as much as on technical innovation.
"""

# ✂️ Split text into 13 chunks
def chunk_text(text, num_chunks=13):
    words = text.split()
    chunk_size = len(words) // num_chunks
    return [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

chunks = chunk_text(text, num_chunks=13)
print(f"✅ Total chunks created: {len(chunks)}")


✅ Total chunks created: 14


In [ ]:
# 🔢 Create embeddings
embeddings = model.encode(chunks, show_progress_bar=True)

# 🧩 Prepare vectors for Pinecone
to_upsert = [(f"id-{i}", emb.tolist(), {"text": chunk}) for i, (chunk, emb) in enumerate(zip(chunks, embeddings))]

# ⬆️ Upsert to Pinecone
index.upsert(vectors=to_upsert)
print("✅ Data upserted to Pinecone!")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Data upserted to Pinecone!


In [ ]:
import time

queries = [
    "What are the ethical concerns with autonomous drones?",
    "How did drones impact civilians in Ukraine?",
    "What are engineers' responsibilities in military projects?",
    "Compare Shahed-136 and Bayraktar TB2 drones",
    "What are the psychological effects of drone warfare?"
]

for q in queries:
    print(f"\n🧠 Query: {q}")

    start_time = time.time()

    # Measure embedding time
    t0 = time.time()
    query_embedding = model.encode([q])[0].tolist()
    embedding_time = time.time() - t0

    # Measure query time
    t1 = time.time()
    result = index.query(vector=query_embedding, top_k=3, include_metadata=True)
    query_time = time.time() - t1

    total_time = time.time() - start_time

    # Display results
    print(f"⚙️ Embedding Time: {embedding_time:.3f}s | Query Time: {query_time:.3f}s | Total: {total_time:.3f}s")

    for i, match in enumerate(result["matches"], 1):
        text = match["metadata"]["text"]
        score = match["score"]
        print(f"\n🔹 Rank {i} — Similarity: {score:.4f}")
        print(f"{text[:300]}...")  # Show preview of chunk



🧠 Query: What are the ethical concerns with autonomous drones?
⚙️ Embedding Time: 0.054s | Query Time: 0.051s | Total: 0.105s

🔹 Rank 1 — Similarity: 0.7068
take away that engineering in warfare is never neutral: every technical choice has social and moral implications. As autonomous weapons proliferate, engineers must engage proactively in ethical reflection and policymaking. Ensuring that drone technology serves defensive security goals without underm...

🔹 Rank 2 — Similarity: 0.6929
. The combination of low cost and high lethality of drone swarms is creating a new paradigm. Combatants and civilians alike see their world transformed by night-time drone swarms. Society must ask: should restrictions be placed on fully autonomous swarms? Should there be an international treaty limi...

🔹 Rank 3 — Similarity: 0.6852
in technology and warfare. When people see autonomous or remotely piloted weapons causing civilian suffering, they may lose faith in institutions and engineers who develop 